### **[LeetCode Link](https://leetcode-cn.com/problems/maximum-subarray/solution/dong-tai-gui-hua-fen-zhi-fa-python-dai-ma-java-dai/)**

## 动态规划
### 第 1 步：定义状态
既然一个连续子数组一定要以一个数作为结尾，那么我们就将状态定义成如下。
* `dp[i]`：表示以 `nums[i]` 结尾的连续子数组的最大和。

### 第 2 步：思考状态转移方程
根据状态的定义，由于 `nums[i]` 一定会被选取，并且 `dp[i]` 所表示的连续子序列与 `dp[i - 1]` 所表示的连续子序列（有可能）就差一个 `nums[i]` 。

假设数组 `nums` 全是正数，那么一定有 `dp[i] = dp[i - 1] + nums[i]`。

在一般情况下 `dp[i - 1]` 有可能是负数，例如前几个数都是负数，突然来了一个正数。

于是分类讨论：
* 如果 `dp[i - 1] >= 0`，那么可以把 `nums[i]` 直接接在 `dp[i - 1]` 表示的那个数组的后面。
* 如果 `dp[i - 1] < 0`，那么加上前面的数反而越来越小了，于是“另起炉灶”，单独的一个 `nums[i]`，就是 `dp[i]`。

以上两种情况的最大值就是 `dp[i]` 的值，写出如下状态转移方程：
<center>$d p[i]=\left\{\begin{array}{ll}d p[i-1]+n u m s[i], & \text { if } \quad d p[i-1] \geq 0 \\ n u m s[i], & i f \quad d p[i-1]<0\end{array}\right.$</center>
记为“状态转移方程 1”。

状态转移方程还可以这样写，反正求的是最大值，也不用分类讨论了，就这两种情况，取最大即可，因此还可以写出状态转移方程如下：
<center>$dp[i] = \max \{nums[i],\; dp[i - 1] + nums[i]\}$</center>
记为“状态转移方程 2”。

> 动态规划的问题经常要分类讨论，这是因为动态规划的问题本来就有最优子结构的特征，即大问题的最优解通常由小问题的最优解得到，那么我们就需要通过分类讨论，得到大问题的小问题究竟是哪些。

### 第 3 步：思考初始值
`dp[0]` 根据定义，一定以 `nums[0]` 结尾，因此 `dp[0] = nums[0]`。

### 第 4 步：思考输出
这里状态的定义不是题目中的问题的定义，不能直接将最后一个状态返回回去。

输出应该是把所有的 `dp[0]`、`dp[1]`、……、`dp[n - 1]` 都看一遍，取最大值。 

### 第 5 步：思考优化空间
既然当前状态只与上一个状态有关，我们可以将空间复杂度降到 $O(1)$。

### 复杂度分析：
* 时间复杂度：$O(N)$。
* 空间复杂度：$O(N)$。

In [ ]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        totall=[nums[0]]
        for i in range(1,len(nums)):
            if totall[i-1]<=0:
                totall.append(nums[i])
            else:
                totall.append(nums[i]+totall[i-1])
        return max(totall)

## 分治
### 思路和算法
我们定义一个操作 `get(a, l, r)` 表示查询 $a$ 序列 $[l, r]$ 区间内的最大子段和，那么最终我们要求的答案就是 `get(nums, 0, nums.size() - 1)`。如何分治实现这个操作呢？对于一个区间 `[l, r]`，我们取 $m = \lfloor \frac{l + r}{2} \rfloor$，对区间 $[l, m]$ 和 $[m + 1, r]$ 分治求解。当递归逐层深入直到区间长度缩小为 $1$ 的时候，递归「开始回升」。这个时候我们考虑如何通过 $[l, m]$ 区间的信息和 $[m + 1, r]$ 区间的信息合并成区间 $[l, r]$ 的信息。最关键的两个问题是：
* 我们要维护区间的哪些信息呢？
* 我们如何合并这些信息呢？

对于一个区间 $[l, r]$，我们可以维护四个量：
* `lSum` 表示 $[l, r]$ 内以 $l$ 为左端点的最大子段和
* `rSum` 表示 $[l, r]$ 内以 $r$ 为右端点的最大子段和
* `mSum` 表示 $[l, r]$ 内的最大子段和
* `iSum` 表示 $[l, r]$ 的区间和

以下简称 $[l, m]$ 为 $[l, r]$ 的「左子区间」，$[m + 1, r]$ 为 $[l, r]$ 的「右子区间」。我们考虑如何维护这些量呢（如何通过左右子区间的信息合并得到 $[l, r]$ 的信息）？对于长度为 $1$ 的区间 $[i, i]$，四个量的值都和 $a_i$ 相等。对于长度大于 $1$ 的区间：
* 首先最好维护的是 `iSum`，区间 $[l, r]$ 的 `iSum` 就等于「左子区间」的 `iSum` 加上「右子区间」的 `iSum`。
* 对于 $[l, r]$ 的 `lSum`，存在两种可能，它要么等于「左子区间」的 `lSum`，要么等于「左子区间」的 `iSum` 加上「右子区间」的 `lSum`，二者取大。
* 对于 $[l, r]$ 的 `rSum`，同理，它要么等于「右子区间」的 `rSum`，要么等于「右子区间」的 `iSum` 加上「左子区间」的 `rSum`，二者取大。
* 当计算好上面的三个量之后，就很好计算 $[l, r]$ 的 `mSum` 了。我们可以考虑 $[l, r]$ 的 `mSum` 对应的区间是否跨越 $m$——它可能不跨越 $m$，也就是说 $[l, r]$ 的 `mSum` 可能是「左子区间」的 `mSum` 和 「右子区间」的 `mSum` 中的一个；它也可能跨越 $m$，可能是「左子区间」的 `rSum` 和 「右子区间」的 `lSum` 求和。三者取大。

### 复杂度分析
假设序列 $a$ 的长度为 $n$。
* 时间复杂度：假设我们把递归的过程看作是一颗二叉树的先序遍历，那么这颗二叉树的深度的渐进上界为 $O(\log n)$，这里的总时间相当于遍历这颗二叉树的所有节点，故总时间的渐进上界是 $O(\sum_{i = 1}^{\log n} 2^{i - 1}) = O(n)$，故渐进时间复杂度为 $O(n)$。
* 空间复杂度：递归会使用 $O(\log n)$ 的栈空间，故渐进空间复杂度为 $O(\log n)$。

In [ ]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        size = len(nums)
        if size == 0:
            return 0
        return self.__max_sub_array(nums, 0, size - 1)

    def __max_sub_array(self, nums, left, right):
        if left == right:
            return nums[left]
        mid = (left + right) >> 1
        return max(self.__max_sub_array(nums, left, mid),
                   self.__max_sub_array(nums, mid + 1, right),
                   self.__max_cross_array(nums, left, mid, right))

    def __max_cross_array(self, nums, left, mid, right):
        # 一定包含 nums[mid] 元素的最大连续子数组的和，
        # 思路是看看左边"扩散到底"，得到一个最大数，右边"扩散到底"得到一个最大数
        # 然后再加上中间数
        left_sum_max = 0
        start_left = mid - 1
        s1 = 0
        while start_left >= left:
            s1 += nums[start_left]
            left_sum_max = max(left_sum_max, s1)
            start_left -= 1

        right_sum_max = 0
        start_right = mid + 1
        s2 = 0
        while start_right <= right:
            s2 += nums[start_right]
            right_sum_max = max(right_sum_max, s2)
            start_right += 1
        return left_sum_max + nums[mid] + right_sum_max